# Load All libraries

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split,GridSearchCV, StratifiedKFold
from xgboost import XGBClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score



from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Load the data set

In [2]:
# Load a dataset into a Pandas Dataframe
train_df = pd.read_csv('./spaceship-titanic/Kaggle_Competition_Data/train.csv')
test_df = pd.read_csv('./spaceship-titanic/Kaggle_Competition_Data/test.csv')
print("Full train dataset shape is {}".format(train_df.shape))
print("Full test dataset shape is {}".format(test_df.shape))

Full train dataset shape is (8693, 14)
Full test dataset shape is (4277, 13)


# EDA 

In [3]:
# Display the first 5 examples
train_df.head(5)

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0001_01     Europa     False  B/0/P  TRAPPIST-1e 39.00  False   
1     0002_01      Earth     False  F/0/S  TRAPPIST-1e 24.00  False   
2     0003_01     Europa     False  A/0/S  TRAPPIST-1e 58.00   True   
3     0003_02     Europa     False  A/0/S  TRAPPIST-1e 33.00  False   
4     0004_01      Earth     False  F/1/S  TRAPPIST-1e 16.00  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck               Name  \
0         0.00       0.00          0.00    0.00    0.00    Maham Ofracculy   
1       109.00       9.00         25.00  549.00   44.00       Juanna Vines   
2        43.00    3576.00          0.00 6715.00   49.00      Altark Susent   
3         0.00    1283.00        371.00 3329.00  193.00       Solam Susent   
4       303.00      70.00        151.00  565.00    2.00  Willy Santantines   

   Transported  
0        False  
1         True  
2        False  
3        False  
4         True

In [4]:
test_df.head(5)

PassengerId HomePlanet CryoSleep  Cabin  Destination   Age    VIP  \
0     0013_01      Earth      True  G/3/S  TRAPPIST-1e 27.00  False   
1     0018_01      Earth     False  F/4/S  TRAPPIST-1e 19.00  False   
2     0019_01     Europa      True  C/0/S  55 Cancri e 31.00  False   
3     0021_01     Europa     False  C/1/S  TRAPPIST-1e 38.00  False   
4     0023_01      Earth     False  F/5/S  TRAPPIST-1e 20.00  False   

   RoomService  FoodCourt  ShoppingMall     Spa  VRDeck              Name  
0         0.00       0.00          0.00    0.00    0.00   Nelly Carsoning  
1         0.00       9.00          0.00 2823.00    0.00    Lerome Peckers  
2         0.00       0.00          0.00    0.00    0.00   Sabih Unhearfus  
3         0.00    6652.00          0.00  181.00  585.00  Meratz Caltilter  
4        10.00       0.00        635.00    0.00    0.00   Brence Harperez

In [5]:
train_df.describe()

Age  RoomService  FoodCourt  ShoppingMall      Spa   VRDeck
count 8514.00      8512.00    8510.00       8485.00  8510.00  8505.00
mean    28.83       224.69     458.08        173.73   311.14   304.85
std     14.49       666.72    1611.49        604.70  1136.71  1145.72
min      0.00         0.00       0.00          0.00     0.00     0.00
25%     19.00         0.00       0.00          0.00     0.00     0.00
50%     27.00         0.00       0.00          0.00     0.00     0.00
75%     38.00        47.00      76.00         27.00    59.00    46.00
max     79.00     14327.00   29813.00      23492.00 22408.00 24133.00

In [6]:
test_df.describe()

Age  RoomService  FoodCourt  ShoppingMall      Spa   VRDeck
count 4186.00      4195.00    4171.00       4179.00  4176.00  4197.00
mean    28.66       219.27     439.48        177.30   303.05   310.71
std     14.18       607.01    1527.66        560.82  1117.19  1246.99
min      0.00         0.00       0.00          0.00     0.00     0.00
25%     19.00         0.00       0.00          0.00     0.00     0.00
50%     26.00         0.00       0.00          0.00     0.00     0.00
75%     37.00        53.00      78.00         33.00    50.00    36.00
max     79.00     11567.00   25273.00       8292.00 19844.00 22272.00

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [9]:
plot_df = train_df.Transported.value_counts()
plot_df.plot(kind="bar")

<Axes: >

In [10]:
# fig, ax = plt.subplots(5,1,  figsize=(10, 10))
# plt.subplots_adjust(top = 2)

# sns.histplot(dataset_df['Age'], color='b', bins=50, ax=ax[0])
# sns.histplot(dataset_df['FoodCourt'], color='b', bins=50, ax=ax[1])
# sns.histplot(dataset_df['ShoppingMall'], color='b', bins=50, ax=ax[2])
# sns.histplot(dataset_df['Spa'], color='b', bins=50, ax=ax[3])
# sns.histplot(dataset_df['VRDeck'], color='b', bins=50, ax=ax[4])

In [11]:
# check for null values
train_df.isnull().sum().sort_values(ascending=False)

CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64

In [12]:
# check for null values
test_df.isnull().sum().sort_values(ascending=False)

FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
VRDeck           80
PassengerId       0
dtype: int64

In [13]:
print("Train unique values: \n",train_df.nunique() )

Train unique values: 
 PassengerId     8693
HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64


In [14]:
print("test unique values: \n",test_df.nunique() )

test unique values: 
 PassengerId     4277
HomePlanet         3
CryoSleep          2
Cabin           3265
Destination        3
Age               79
VIP                2
RoomService      842
FoodCourt        902
ShoppingMall     715
Spa              833
VRDeck           796
Name            4176
dtype: int64


In [15]:
print("Train distinct values: \n",train_df['HomePlanet'].unique())

Train distinct values: 
 ['Europa' 'Earth' 'Mars' nan]


In [16]:
print("test distinct values: \n",test_df['HomePlanet'].unique())

test distinct values: 
 ['Earth' 'Europa' 'Mars' nan]


In [17]:
# List of unique values 'Destination'
print("Train distinct values: \n",train_df['Destination'].unique())

Train distinct values: 
 ['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]


In [18]:
# List of unique values 'Destination'
print("test distinct values: \n",test_df['Destination'].unique())

test distinct values: 
 ['TRAPPIST-1e' '55 Cancri e' 'PSO J318.5-22' nan]


In [19]:
# Check for duplicate Ids
# Train
idsUnique = len(set(train_df.PassengerId))
idsTotal = train_df.shape[0]
idsDupli = idsTotal - idsUnique
print("Train: There are " + str(idsDupli) + " duplicate IDs for " + str(idsTotal) + " total entries")

Train: There are 0 duplicate IDs for 8693 total entries


In [20]:
 # we drop PassengerId and Name as they dont provide any valuable meaning to training data 
train = train_df.drop(['PassengerId', 'Name'], axis=1)
train.head(5)

HomePlanet CryoSleep  Cabin  Destination   Age    VIP  RoomService  \
0     Europa     False  B/0/P  TRAPPIST-1e 39.00  False         0.00   
1      Earth     False  F/0/S  TRAPPIST-1e 24.00  False       109.00   
2     Europa     False  A/0/S  TRAPPIST-1e 58.00   True        43.00   
3     Europa     False  A/0/S  TRAPPIST-1e 33.00  False         0.00   
4      Earth     False  F/1/S  TRAPPIST-1e 16.00  False       303.00   

   FoodCourt  ShoppingMall     Spa  VRDeck  Transported  
0       0.00          0.00    0.00    0.00        False  
1       9.00         25.00  549.00   44.00         True  
2    3576.00          0.00 6715.00   49.00        False  
3    1283.00        371.00 3329.00  193.00        False  
4      70.00        151.00  565.00    2.00         True

In [21]:
# Test
idsUnique = len(set(test_df.PassengerId))
idsTotal = test_df.shape[0]
idsDupli = idsTotal - idsUnique
print("Test: There are " + str(idsDupli) + " duplicate IDs for " + str(idsTotal) + " total entries")

Test: There are 0 duplicate IDs for 4277 total entries


In [22]:
 # we drop PassengerId and Name as they dont provide any valuable meaning to training data 
test = test_df.drop(['PassengerId', 'Name'], axis=1)
test.head(5)

HomePlanet CryoSleep  Cabin  Destination   Age    VIP  RoomService  \
0      Earth      True  G/3/S  TRAPPIST-1e 27.00  False         0.00   
1      Earth     False  F/4/S  TRAPPIST-1e 19.00  False         0.00   
2     Europa      True  C/0/S  55 Cancri e 31.00  False         0.00   
3     Europa     False  C/1/S  TRAPPIST-1e 38.00  False         0.00   
4      Earth     False  F/5/S  TRAPPIST-1e 20.00  False        10.00   

   FoodCourt  ShoppingMall     Spa  VRDeck  
0       0.00          0.00    0.00    0.00  
1       9.00          0.00 2823.00    0.00  
2       0.00          0.00    0.00    0.00  
3    6652.00          0.00  181.00  585.00  
4       0.00        635.00    0.00    0.00

In [23]:
# Check for missing values
# Train
train_missing_val_count = (train.isnull().sum())
# Test
test_missing_val_count = (test.isnull().sum())
print("Train missing values: \n",train_missing_val_count[train_missing_val_count>0],"\n\n","Test missing values: \n",  test_missing_val_count[test_missing_val_count>0])

Train missing values: 
 HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
dtype: int64 

 Test missing values: 
 HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
dtype: int64


# Preprocessing

In [24]:
imputer_cols = ["Age", "FoodCourt", "ShoppingMall", "Spa", "VRDeck" ,"RoomService"]
imputer = SimpleImputer(strategy="median" )
imputer.fit(train[imputer_cols])
train[imputer_cols] = imputer.transform(train[imputer_cols])
test[imputer_cols] = imputer.transform(test[imputer_cols])
train["HomePlanet"].fillna('Z', inplace=True)
test["HomePlanet"].fillna('Z', inplace=True)

In [25]:
label_cols = ["HomePlanet", "CryoSleep","Cabin", "Destination" ,"VIP"]
def label_encoder(train,test,columns):
    for col in columns:
        train[col] = train[col].astype(str)
        test[col] = test[col].astype(str)
        train[col] = LabelEncoder().fit_transform(train[col])
        test[col] =  LabelEncoder().fit_transform(test[col])
    return train, test

train ,test = label_encoder(train,test ,label_cols)

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8693 non-null   int32  
 1   CryoSleep     8693 non-null   int32  
 2   Cabin         8693 non-null   int32  
 3   Destination   8693 non-null   int32  
 4   Age           8693 non-null   float64
 5   VIP           8693 non-null   int32  
 6   RoomService   8693 non-null   float64
 7   FoodCourt     8693 non-null   float64
 8   ShoppingMall  8693 non-null   float64
 9   Spa           8693 non-null   float64
 10  VRDeck        8693 non-null   float64
 11  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), int32(5)
memory usage: 585.9 KB


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    4277 non-null   int32  
 1   CryoSleep     4277 non-null   int32  
 2   Cabin         4277 non-null   int32  
 3   Destination   4277 non-null   int32  
 4   Age           4277 non-null   float64
 5   VIP           4277 non-null   int32  
 6   RoomService   4277 non-null   float64
 7   FoodCourt     4277 non-null   float64
 8   ShoppingMall  4277 non-null   float64
 9   Spa           4277 non-null   float64
 10  VRDeck        4277 non-null   float64
dtypes: float64(6), int32(5)
memory usage: 284.1 KB


# splitting training data 

In [28]:
train.drop(["Cabin"] , axis = 1 ,inplace = True)
test.drop(["Cabin"] , axis = 1 ,inplace = True)
X = train.drop("Transported" , axis =1 )
y = train["Transported"]
X_train , X_val , y_train , y_val = train_test_split(X , 
                                                       y, 
                                                       random_state = 0,
                                                       test_size =0.2)



# creating models and using grid search CV on all to get best models 

In [29]:
# clf = LazyClassifier(verbose=0,
#                      ignore_warnings=True,
#                      custom_metric=None,
#                      predictions=False,
#                      random_state=0,
#                      classifiers='all')

# models, predictions = clf.fit(X_train , X_val , y_train , y_val)

In [30]:
# models[:15]

In [31]:
# xgb = XGBClassifier(booster='gbtree', tree_method='gpu_hist', gpu_id=0,nthread=6)
# param_grid = {
#      'max_depth': [6,7,8,9,10,12],
#     'n_estimators': [200, 300,400],
#     'learning_rate': [0.01, 0.05,0.02,0.03],
# }

# scoring = ['f1']
# # Set up the k-fold cross-validation
        
# grid_search = GridSearchCV(estimator=xgb, 
#                    param_grid=param_grid, 
#                    scoring=scoring, 
#                    refit='f1', 
#                    n_jobs=6, 
#                    cv=3, 
#                    verbose=3,
#                           )
# # Fit grid search
# grid_search.fit(X_train, y_train)
# xgb_best_model = grid_search.best_estimator_

In [32]:
# print("best params", grid_search.best_params_)
# best params {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 400}

In [33]:
# Define your six base models
model1 = LogisticRegression()
model2 = GaussianNB()
model3 = LinearDiscriminantAnalysis()
model4 = DecisionTreeClassifier(random_state = 0)
model5 = RandomForestClassifier(random_state = 0)
model6 = GradientBoostingClassifier(random_state = 0)

# Train the base models on the training data
model1.fit(X_train, y_train)
model2.fit(X_train, y_train)
model3.fit(X_train, y_train)
model4.fit(X_train, y_train)
model5.fit(X_train, y_train)
model6.fit(X_train, y_train)

# Use the base models to make predictions on the validation set
preds1 = model1.predict(X_val)
preds2 = model2.predict(X_val)
preds3 = model3.predict(X_val)
preds4 = model4.predict(X_val)
preds5 = model5.predict(X_val)
preds6 = model6.predict(X_val)

# Combine the predictions into a new dataset
meta_X = np.column_stack((preds1, preds2, preds3, preds4, preds5, preds6))

# Define the meta-model (logistic regression in this case)
meta_model = LogisticRegression()

# Train the meta-model on the combined predictions
meta_model.fit(meta_X, y_val)

# Use the base models to make predictions on the entire dataset
base_preds1 = model1.predict(test)
base_preds2 = model2.predict(test)
base_preds3 = model3.predict(test)
base_preds4 = model4.predict(test)
base_preds5 = model5.predict(test)
base_preds6 = model6.predict(test)

# Combine the base model predictions into a new dataset
meta_X_test = np.column_stack((base_preds1, base_preds2, base_preds3, base_preds4, base_preds5, base_preds6))

# Use the meta-model to make predictions on the test set
meta_preds = meta_model.predict(meta_X_test)

# Output predictions in the required format
# output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived': meta_preds})
# output.to_csv('predictions.csv', index=False)

output = pd.DataFrame(test_df['PassengerId'])
output['Transported'] = meta_preds
output['Transported'] = output['Transported'].replace({1: True, 0: False})
output.to_csv('predictions.csv', index=False)


In [34]:
len(output['Transported'] )

4277

## Logistic regression

In [35]:


logReg_accuracy = round(accuracy_score(y_val, preds1), 3)
logReg_precision = round(precision_score(y_val, preds1), 3)
logReg_recall = round(recall_score(y_val, preds1), 3)
logReg_f1 = round(f1_score(y_val, preds1, average = 'macro'), 3)
print('-- Accuracy: {} / Precision: {} / Recall: {} / f1: {}'.format(logReg_accuracy,logReg_precision,logReg_recall,logReg_f1))

-- Accuracy: 0.775 / Precision: 0.745 / Recall: 0.84 / f1: 0.773


## GaussianNB

In [36]:



gauss_accuracy = round(accuracy_score(y_val, preds2), 3)
gauss_precision = round(precision_score(y_val, preds2), 3)
gauss_recall = round(recall_score(y_val, preds2), 3)
gauss_f1 = round(f1_score(y_val, preds2, average = 'macro'), 3)
print('-- Accuracy: {} / Precision: {} / Recall: {} / f1: {}'.format(gauss_accuracy,gauss_precision,gauss_recall,gauss_f1))


-- Accuracy: 0.688 / Precision: 0.628 / Recall: 0.935 / f1: 0.667


## LinearDiscriminantAnalysis

In [37]:


lda_accuracy = round(accuracy_score(y_val, preds3), 3)
lda_precision = round(precision_score(y_val, preds3), 3)
lda_recall = round(recall_score(y_val, preds3), 3)
lda_f1 = round(f1_score(y_val, preds3, average = 'macro'), 3)
print('-- Accuracy: {} / Precision: {} / Recall: {} / f1: {}'.format(lda_accuracy,lda_precision,lda_recall,lda_f1))


-- Accuracy: 0.751 / Precision: 0.771 / Recall: 0.719 / f1: 0.751


## DecisionTreeClassifier

In [38]:

dtc_accuracy = round(accuracy_score(y_val, preds4), 3)
dtc_precision = round(precision_score(y_val, preds4), 3)
dtc_recall = round(recall_score(y_val, preds4), 3)
dtc_f1 = round(f1_score(y_val, preds4, average = 'macro'), 3)
print('-- Accuracy: {} / Precision: {} / Recall: {} / f1: {}'.format(dtc_accuracy,dtc_precision,dtc_recall,dtc_f1))


-- Accuracy: 0.723 / Precision: 0.701 / Recall: 0.788 / f1: 0.722


## RandomForestClassifier

In [39]:

rfc_accuracy = round(accuracy_score(y_val, preds5), 3)
rfc_precision = round(precision_score(y_val, preds5), 3)
rfc_recall = round(recall_score(y_val, preds5), 3)
rfc_f1 = round(f1_score(y_val, preds5, average = 'macro'), 3)
print('-- Accuracy: {} / Precision: {} / Recall: {} / f1: {}'.format(rfc_accuracy,rfc_precision,rfc_recall,rfc_f1))


-- Accuracy: 0.785 / Precision: 0.783 / Recall: 0.792 / f1: 0.785


## GradientBoostingClassifier

In [40]:

gbc_accuracy = round(accuracy_score(y_val, preds6), 3)
gbc_precision = round(precision_score(y_val, preds6), 3)
gbc_recall = round(recall_score(y_val, preds6), 3)
gbc_f1 = round(f1_score(y_val, preds6, average = 'macro'), 3)
print('-- Accuracy: {} / Precision: {} / Recall: {} / f1: {}'.format(gbc_accuracy,gbc_precision,gbc_recall,gbc_f1))


-- Accuracy: 0.783 / Precision: 0.762 / Recall: 0.828 / f1: 0.783
